#### This notebook cleans raw data and generate processed data files for further analysis

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import operator 

In [3]:
# Data import 
# trade = pd.read_csv('/Users/Sabrina/Documents/OneDrive/Github/crop-trends/data/raw/Trade_DetailedTradeMatrix_E_All_Data_NOFLAG.csv',
#                    encoding='latin1')


In [4]:
# Try normalized data?
trade = pd.read_csv('/Users/Sabrina/Downloads/Trade_DetailedTradeMatrix_E_All_Data_Normalized/Trade_DetailedTradeMatrix_E_All_Data_Normalized.csv',
                   encoding='latin1')


## 1. Inspect general information of the dataset

In [5]:
print(trade.shape) # total rows and columns 

(35976124, 13)


In [6]:
print(trade.shape)

(35976124, 13)


In [9]:
trade.sample(5) # time series between years 1986 and 2017 on export and import agricultural commodities.

,Reporter Country Code,Reporter Countries,Partner Country Code,Partner Countries,Item Code,Item,Element Code,Element,Year Code,Year,Unit,Value,Flag
27702338,200,Singapore,237,Viet Nam,220,Chestnut,5910,Export Quantity,2017,2017,tonnes,1.0,Im
5856829,96,"China, Hong Kong SAR",216,Thailand,1164,"Meat, dried nes",5610,Import Quantity,1993,1993,tonnes,2.0,NaN
24609836,173,Poland,162,Norway,892,"Yoghurt, concentrated or not",5910,Export Quantity,2017,2017,tonnes,1.0,NaN
30395509,210,Sweden,229,United Kingdom,892,"Yoghurt, concentrated or not",5922,Export Value,1989,1989,1000 US$,59.0,NaN
17445396,110,Japan,151,Netherlands Antilles (former),828,Cigarettes,5922,Export Value,1995,1995,1000 US$,1047.0,NaN


In [11]:
# Identify total number of reporter countries, partner countries and unique items
print('Number of unique items traded:', len(trade.Item.unique()))
print('Number of Reporter Countries:', len(trade['Reporter Countries'].unique()))
print('Number of Partner Countries:', len(trade['Partner Countries'].unique()))
print('Entries with Unspecified Area as partner countries:',
      len(trade.loc[trade['Partner Countries']=='Unspecified Area',:]))

Number of unique items traded: 424
Number of Reporter Countries: 184
Number of Partner Countries: 255
Entries with Unspecified Area as partner countries: 86692


In [21]:
trade.loc[trade['Value'].isnull(),:]

,Reporter Country Code,Reporter Countries,Partner Country Code,Partner Countries,Item Code,Item,Element Code,Element,Year Code,Year,Unit,Value,Flag
8191951,167,Czechia,150,Netherlands,866,Cattle,5908,Export Quantity,1995,1995,Head,NaN,M


In [25]:
trade = trade.dropna(subset=['Value']) # Remove missing values in 'Value' colum
print(trade.shape)

(35976123, 13)


In [6]:
# Identify total number of reporter countries, partner countries and unique items
print('Number of unique items traded:', len(trade.Item.unique()))
print('Number of Reporter Countries:', len(trade['Reporter Countries'].unique()))
print('Number of Partner Countries:', len(trade['Partner Countries'].unique()))
print('Entries with Unspecified Area as partner countries:',
      len(trade.loc[trade['Partner Countries']=='Unspecified Area',:]))

Number of unique items traded: 424
Number of Reporter Countries: 184
Number of Partner Countries: 255
Entries with Unspecified Area as partner countries: 399242


 ### Observations
 - Missing values in the ‘Value’ column were checked and one row which contains missing data was dropped. Most missing values in this datafile has been filled by imputation with partner country data
 - 'Partner Countries' include entries of 'Unspecified Area', which should be noted when calculating diversity of trading partners. 
 
 We are not interested in data of a particular item between two specific countries. Instead, we are looking for the number of items(i.e., trading diversity) and their total amount a country export/import, and the number of partner countries each reporter country have for an item.   
 Therefore, we shall aggregate the dataset based on items and reporter countries, respectively. 

## 2. Aggregate data according to items and partner countries

#### 2.1 Sum up trading volumn for reporter countries by items

In [26]:
items_by_country = trade.groupby(['Reporter Countries','Item','Element','Unit','Year Code','Year']).sum()

In [15]:
items_by_country = items_by_country.drop(['Reporter Country Code','Partner Country Code','Element Code'],axis=1)

In [21]:
items_by_country.shape

(223049, 33)

In [20]:
items_by_country.shape

(3399979, 8)

In [18]:
items_by_country = items_by_country.reset_index()

In [19]:
items_by_country.head()

,Reporter Countries,Item,Element,Unit,Year Code,Year,Item Code,Value
0,Afghanistan,Almonds shelled,Export Quantity,tonnes,2009,2009,1155,4763.0
1,Afghanistan,Almonds shelled,Export Quantity,tonnes,2010,2010,1848,1308.0
2,Afghanistan,Almonds shelled,Export Quantity,tonnes,2011,2011,1848,2261.0
3,Afghanistan,Almonds shelled,Export Quantity,tonnes,2014,2014,3003,2714.0
4,Afghanistan,Almonds shelled,Export Quantity,tonnes,2015,2015,2541,2086.0


In [12]:
items_by_country.head()

,index,Reporter Countries,Item,Element,Unit,Item Code,Year Code,Year,Value
0,0,Afghanistan,Almonds shelled,Export Quantity,tonnes,14784,128873,128873,17666.0
1,1,Afghanistan,Almonds shelled,Export Value,1000 US$,14784,128873,128873,131498.0
2,2,Afghanistan,Almonds shelled,Import Quantity,tonnes,3234,28218,28218,1298.0
3,3,Afghanistan,Almonds shelled,Import Value,1000 US$,3234,28218,28218,8241.0
4,4,Afghanistan,"Almonds, with shell",Export Quantity,tonnes,8840,80527,80527,18797.0


In [29]:
len(trade['Year'].unique())

32

#### 2.2 Sum up trading volumn for each reporter countries by partner countries

In [21]:
partners_by_country = trade.groupby(['Reporter Countries','Partner Countries','Element','Unit','Year Code','Year']).sum().reset_index()

In [22]:
# Sum of tonnes, heads, etc across different items do not make much sense, so will be removed
# Only value amount (i.e., US$ will be kept)
partners_by_country = partners_by_country.loc[partners_by_country['Unit']=='1000 US$',:]

In [23]:
partners_by_country = partners_by_country.drop(['Reporter Country Code','Partner Country Code','Item Code','Element Code'],axis=1)

In [37]:
partners_by_country.shape

(57838, 36)

In [38]:
partners_by_country.head()

,Reporter Countries,Partner Countries,Element,Unit,Y1986,Y1987,Y1988,Y1989,Y1990,Y1991,...,Y2008,Y2009,Y2010,Y2011,Y2012,Y2013,Y2014,Y2015,Y2016,Y2017
1,Afghanistan,Algeria,Export Value,1000 US$,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,27.0,1.0,24.0,5.0
3,Afghanistan,Angola,Export Value,1000 US$,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
5,Afghanistan,Argentina,Import Value,1000 US$,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,415.0,0.0,0.0,1163.0,159.0,1844.0,1829.0
7,Afghanistan,Armenia,Export Value,1000 US$,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
9,Afghanistan,Armenia,Import Value,1000 US$,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,48.0,0.0,0.0,282.0,246.0,0.0,67.0


In [24]:
partners_by_country.shape

(754206, 7)

In [25]:
partners_by_country.head()

,Reporter Countries,Partner Countries,Element,Unit,Year Code,Year,Value
2,Afghanistan,Algeria,Export Value,1000 US$,2014,2014,27.0
3,Afghanistan,Algeria,Export Value,1000 US$,2015,2015,1.0
4,Afghanistan,Algeria,Export Value,1000 US$,2016,2016,24.0
5,Afghanistan,Algeria,Export Value,1000 US$,2017,2017,5.0
7,Afghanistan,Angola,Export Value,1000 US$,2015,2015,1.0


In [39]:
# Export aggregated data
items_by_country.to_csv('../data/processed/items_by_country.csv')
partners_by_country.to_csv('../data/processed/partners_by_country.csv')